In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.model_selection import train_test_split

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(128,128)).numpy()
x_test=tf.image.resize(x_test,(128,128)).numpy()

In [5]:
x_train=tf.keras.applications.mobilenet.preprocess_input(x_train)
x_test=tf.keras.applications.mobilenet.preprocess_input(x_test)

In [6]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model=MobileNetV3Small(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

/usr/local/lib/python3.10/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
base_model.trainable=False

In [10]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)        │ (None, 4, 4, 576)           │         939,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 576)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         590,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,218,964 (8.46 MB)

 Trainable params: 1,276,260 (4.87 MB)

 Non-trainable params: 942,704 (3.60 MB)

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger
early_stopper=EarlyStopping(patience=5,monitor="val_loss",restore_best_weights=True)

In [13]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [14]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [15]:
import time
start_time=time.time()

In [16]:
history=model.fit(datagen.flow(x_train,y_train),epochs=30,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 155s 127ms/step - accuracy: 0.0210 - loss: 5.1309 - val_accuracy: 0.0697 - val_loss: 4.2700
Epoch 2/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 112ms/step - accuracy: 0.0493 - loss: 4.5775 - val_accuracy: 0.0905 - val_loss: 4.1034
Epoch 3/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 133s 113ms/step - accuracy: 0.0651 - loss: 4.3524 - val_accuracy: 0.0982 - val_loss: 4.0438
Epoch 4/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 133s 113ms/step - accuracy: 0.0732 - loss: 4.2495 - val_accuracy: 0.1077 - val_loss: 3.9300
Epoch 5/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.0797 - loss: 4.1636 - val_accuracy: 0.1384 - val_loss: 3.7533
Epoch 6/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 133s 113ms/step - accuracy: 0.0870 - loss: 4.0868 - val_accuracy: 0.1470 - val_loss: 3.7110
Epoch 7/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 132s 113ms/step - accuracy: 0.0992 - loss: 4.0163 - val_accuracy: 0.1081 - val_loss: 3.9763
Epoch 8/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 133s 114ms/step - ac

In [17]:
head_training_time=time.time()-start_time

In [18]:
base_model.trainable=True

In [19]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [20]:
start_fine_tune=time.time()

In [21]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 238s 187ms/step - accuracy: 0.0175 - loss: 5.2458 - val_accuracy: 0.0126 - val_loss: 6.0320
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 157s 125ms/step - accuracy: 0.0598 - loss: 4.2319 - val_accuracy: 0.0123 - val_loss: 7.0092
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 147s 125ms/step - accuracy: 0.0982 - loss: 3.8550 - val_accuracy: 0.0186 - val_loss: 5.4398
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 146s 125ms/step - accuracy: 0.1290 - loss: 3.6098 - val_accuracy: 0.0963 - val_loss: 3.9512
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 147s 125ms/step - accuracy: 0.1509 - loss: 3.4482 - val_accuracy: 0.1754 - val_loss: 3.3707
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 147s 126ms/step - accuracy: 0.1779 - loss: 3.3013 - val_accuracy: 0.2490 - val_loss: 3.0044
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 147s 125ms/step - accuracy: 0.2016 - loss: 3.1579 - val_accuracy: 0.3008 - val_loss: 2.7587
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 146s 125ms/step - ac

In [22]:
fine_tune_time=time.time()-start_fine_tune

In [23]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step


In [24]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.3932 - loss: 2.3477

Test Accuracy: 39.24%
Total training time: 4000.11 seconds


In [25]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3846 - loss: 2.3717
Validation accuracy: 38.7840%


In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_class, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred_class, average='weighted', zero_division=1)

print("\nPerformance Metrics Breakdown for MobileNetV3Small:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for MobileNetV3Small:
Accuracy: 0.3924
Precision: 0.3929
Recall: 0.3924
F1-Score: 0.3653


In [27]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["MobileNetV3 Small"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [28]:
print(results)

               Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  MobileNetV3 Small         0.3924              0.38784   2.343888   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0              2422.01728           1578.097682       2218964  


In [29]:
results.to_csv("MobileNetV3Small .csv",index=False)
print("\nResults saved to MobileNetV3Small.csv")


Results saved to MobileNetV3Small.csv
